# Unit Commitment Problem with Machine Learning
In this notebook, we solve a unit commitment problem for a power system using optimization and machine learning. We use binary classification to predict the on/off status of each generator based on system conditions, aiming to accelerate the unit commitment process.

In [14]:
import numpy as np
from gurobipy import Model, GRB, quicksum
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

## Step 1: Define the Unit Commitment Model

                     N1  N2  N3  N4  N5  N6
2019-01-01 00:00:00  10  15  20  25  30  35
2019-01-01 01:00:00  20  25  30  35  40  45
2019-01-01 02:00:00  30  35  40  45  50  55
查找到的值: 25


## Step 2: Data Preparation - Generating Synthetic Data

## Generate Load data based on nominal load with fluctuate

In [143]:
import numpy as np
import pandas as pd

# Define constants
nominal_load_1 = 56
nominal_load_2 = 112
nominal_load_3 = 112
hours_per_year = 365 * 24

# Adding sinusoidal daily cycle and random noise for variability
time = np.arange(hours_per_year)
daily_cycle = 0.2 * np.sin(2 * np.pi * time / 24)  # Daily sinusoidal pattern (20% amplitude)
seasonal_cycle = 0.1 * np.sin(2 * np.pi * time / (24 * 30))  # Monthly sinusoidal pattern (10% amplitude)
seasonal_cycle_2 = 0.1 * np.sin(3 * np.pi * time / (24 * 30)) 
random_noise = np.random.normal(0, 0.05, hours_per_year)  # Random noise
random_noise_3 = np.random.normal(0, 0.1, hours_per_year) 
# Calculate hourly load
hourly_load_1 = nominal_load_1 * (1 + daily_cycle + seasonal_cycle + random_noise)
hourly_load_2 = nominal_load_2 * (1 + daily_cycle + seasonal_cycle_2 + random_noise)
hourly_load_3 = nominal_load_3 * (1 + daily_cycle + seasonal_cycle + random_noise_3)

# Create DataFrame with timestamps
dates = pd.date_range(start="2019-01-01", periods=hours_per_year, freq="H")
load_data = pd.DataFrame({"Timestamp": dates, "Hourly_Load_1": hourly_load_1, "Hourly_Load_2": hourly_load_2, "Hourly_Load_3": hourly_load_3})



## Generate Wind Power data from Website

In [147]:
ninja_wind_1 = pd.read_csv(r"D:\46765\MLA2\Assignment 2\ninja_wind_55.0582_15.1174_10.csv")
ninja_wind_2 = pd.read_csv(r"D:\46765\MLA2\Assignment 2\ninja_wind_55.2402_14.7831_30.csv")

# Rename 'time' column to 'Timestamp' in both datasets for consistency
ninja_wind_1.rename(columns={"time": "Timestamp"}, inplace=True)
ninja_wind_2.rename(columns={"time": "Timestamp"}, inplace=True)

# Convert 'Timestamp' columns to datetime format
ninja_wind_1['Timestamp'] = pd.to_datetime(ninja_wind_1['Timestamp'], format='%Y/%m/%d %H:%M')
ninja_wind_2['Timestamp'] = pd.to_datetime(ninja_wind_2['Timestamp'], format='%Y/%m/%d %H:%M')

# Merging all data based on 'Timestamp'
merged_data = pd.merge(load_data, ninja_wind_1, on="Timestamp", how="outer")
merged_data = pd.merge(merged_data, ninja_wind_2, on="Timestamp", how="outer", suffixes=('_W1', '_W2'))

merged_data['N1'] = 0
merged_data['N2'] = 0
merged_data['N3'] = 0
merged_data['N4'] = merged_data['Hourly_Load_1'] - merged_data['power_W1']
merged_data['N5'] = merged_data['Hourly_Load_2'] - merged_data['power_W2']
merged_data['N6'] = merged_data['Hourly_Load_3']

# Convert specified columns to dictionary format
merged_data_dict = {
    "N1": merged_data["N1"].tolist(),
    "N2": merged_data["N2"].tolist(),
    "N3": merged_data["N3"].tolist(),
    "N4": merged_data["N4"].tolist(),
    "N5": merged_data["N5"].tolist(),
    "N6": merged_data["N6"].tolist()
}


D_nt_df = pd.DataFrame(merged_data_dict, index=dates)

# value = D_nt_df.loc["2019-01-01 01:00:00", "N4"]
print(D_nt_df )



                     N1  N2  N3         N4          N5          N6
2019-01-01 00:00:00   0   0   0  44.960595   80.144191  116.243714
2019-01-01 01:00:00   0   0   0  52.739302   95.669471  109.003829
2019-01-01 02:00:00   0   0   0  49.246561   88.685836  105.381147
2019-01-01 03:00:00   0   0   0  55.233364  100.740256  124.365467
2019-01-01 04:00:00   0   0   0  57.948140  106.224569  140.961975
...                  ..  ..  ..        ...         ...         ...
2019-12-31 19:00:00   0   0   0  46.899106   89.494248  106.988068
2019-12-31 20:00:00   0   0   0  46.717311   89.186134  111.459493
2019-12-31 21:00:00   0   0   0  44.110204   84.029203  111.488070
2019-12-31 22:00:00   0   0   0  45.351600   86.454089  101.468462
2019-12-31 23:00:00   0   0   0  53.168642  101.454077  109.882804

[8760 rows x 6 columns]


## Step 1: Define the Unit Commitment Model

In [ ]:


G = ["G1", "G2", "G3"] 
T = dates
daily_time_blocks = [T[i:i+24] for i in range(0, len(T), 24)]

N = ["N1", "N2", "N3" , "N4" , "N5", "N6"] 
L = ["L1", "L2", "L3" , "L4" , "L5", "L6", "L7"]


# D_nt_df.set_index(['Timestamp', 'N'], inplace=True)
B = [
    [0, -0.68197, -0.65018, -0.48266, -0.51445, -0.63472],
    [0, 0.146043, -0.75311, -0.22164, -0.32249, -0.70405],
    [0, -0.31803, -0.34982, -0.51734, -0.48555, -0.36528],
    [0, 0.17199, 0.102932, -0.26102, -0.19196, 0.069337],
    [0, -0.14604, -0.24689, 0.221642, -0.67751, -0.29595],
    [0, -0.14604, -0.24689, 0.221642, 0.322485, -0.29595],
    [0, 0.146043, 0.246886, -0.22164, -0.32249, -0.70405]
]
max_production = [220, 100, 70]
transmission_capacity = [200, 100, 100, 100, 100, 100, 100]
min_down_time = [4,2,1]
min_production = [100, 10, 10]
min_up_time = [4,3,1]
production_cost = [20.655, 16.7, 22.545]
ramping_rate = [55, 50, 20]
start_up_cost = [900,550,170]
generator_data = pd.DataFrame({
    'Max_Production': max_production,
    'Min_Down_Time': min_down_time,
    'Min_Production': min_production,
    'Min_Up_Time': min_up_time,
    'Production_Cost': production_cost,
    'Ramping_Rate': ramping_rate,
    'Start_Up_Cost': start_up_cost
}, index=G)
B_df = pd.DataFrame(B, index=L, columns=N)
transmission_capacity_df = pd.DataFrame({
    'Transmission_Capacity': transmission_capacity
}, index=L)

Load_1 = merged_data['Hourly_Load_1']
Load_2 = merged_data['Hourly_Load_2']
Load_3 = merged_data['Hourly_Load_3']
P_W1 = merged_data['power_W1']
P_W2 = merged_data['power_W2']


u_values = {}
p_values = {}
daily_min_costs = []


for day, daily_T in enumerate(daily_time_blocks):
    # print(daily_T)
    # D_nt_df.index = pd.to_datetime(D_nt_df.index)
    # daily_T = pd.DatetimeIndex(daily_T)
    # print(type(D_nt_df.index))
    # print(type(daily_T[0])) 
    # print(daily_T[0])
    
    # for n in N:
    #     for t in daily_T:
    #         print(D_nt_df.loc[daily_T[0], n])
   


    model = Model(f"DailyOptimization_Day{day+1}")

    M = 400

    # p, p_nt
    p = model.addVars(G, daily_T, lb=0,name="p")
    p_nt = model.addVars(N, daily_T, lb=0, name="p_nt")
    u = model.addVars(G, daily_T, vtype=GRB.BINARY, name="u")    # on/off
    # u = model.addVars(G, daily_T, lb=0, ub = 1, name="u")
    s = model.addVars(G, daily_T, lb=0, name="s")                # start-up cost
    epsilon = model.addVars(N, daily_T, lb=0, name="epsilon") 
    delta = model.addVars(N, daily_T, lb=0, name="delta")   



    # Objective function
    model.setObjective(
        (quicksum((generator_data.loc[g,'Production_Cost'] * p[g,t] + s[g, t]) for g in G for t in daily_T) + 
        M * quicksum(epsilon[n, t] + delta[n, t] for n in N for t in daily_T)),
        GRB.MINIMIZE
    )
    # Constraints for generation
    for g in G:
        for t in daily_T:
            model.addConstr(generator_data.loc[g, 'Min_Production'] * u[g, t] <= p[g,t])
            model.addConstr(p[g, t] <= generator_data.loc[g, 'Max_Production'] * u[g, t])

    # balancing constraint
    for t in daily_T:
        model.addConstr(quicksum(p[g,t] for g in G) == quicksum(D_nt_df.loc[t, n] + epsilon[n, t] - delta[n, t] for n in N))

    # Transmission limition
    for l in L:
        for t in daily_T:
            model.addConstr(
                                quicksum(B_df.loc[l, n] * (p_nt[n, t] - D_nt_df.loc[t, n] - epsilon[n, t] + delta[n, t]) for n in N) >= 
                                -transmission_capacity_df.loc[l, 'Transmission_Capacity']
                            )
            model.addConstr(
                quicksum(B_df.loc[l, n] * (p_nt[n, t] - D_nt_df.loc[t, n] - epsilon[n, t] + delta[n, t]) for n in N) <= 
                transmission_capacity_df.loc[l, 'Transmission_Capacity']
            )
            
    # combine p_gt with p_nt
    for t in daily_T:
        model.addConstr(p_nt["N1", t] == p["G1", t], name=f"Link_G1_N1_t{t}")
        model.addConstr(p_nt["N2", t] == p["G2", t], name=f"Link_G2_N2_t{t}")
        model.addConstr(p_nt["N6", t] == p["G3", t], name=f"Link_G3_N6_t{t}")
        model.addConstr(p_nt["N3", t] == 0, name=f"Set_N3_Zero_t{t}")
        model.addConstr(p_nt["N4", t] == 0, name=f"Set_N4_Zero_t{t}")
        model.addConstr(p_nt["N5", t] == 0, name=f"Set_N5_Zero_t{t}")


    # start-up constraint
    for g in G:
        for t_idx, t in enumerate(daily_T):
            if t_idx == 0:
                # if there is data in the previous day use the final data of previous day
                if day > 0:
                    prev_day_last_t = daily_time_blocks[day - 1][-1]
                    model.addConstr(s[g, t] >= generator_data.loc[g, 'Start_Up_Cost'] * (u[g, t] - u_values[(g, prev_day_last_t)] ))
                # if there is no data in the previous day, no constraint
            else:
                # if t>0 use data in same day
                prev_t = daily_T[t_idx - 1]
                model.addConstr(s[g, t] >= generator_data.loc[g, 'Start_Up_Cost'] * (u[g, t] - u[g, prev_t]))

    # ramping rate constraint
    for g in G:
        for t_idx, t in enumerate(daily_T):
            if t_idx == 0:
                if day > 0:
                    prev_day_last_t = daily_time_blocks[day - 1][-1]
                    model.addConstr(p[g, t] - p_values[(g, prev_day_last_t)] >= -generator_data.loc[g, 'Ramping_Rate'])
                    model.addConstr(p[g, t] - p_values[(g, prev_day_last_t)] <= generator_data.loc[g, 'Ramping_Rate'])
            else:
                prev_t = daily_T[t_idx - 1]
                print(prev_t)
                model.addConstr(p[g, t] - p[g, prev_t] >= -generator_data.loc[g, 'Ramping_Rate'])
                model.addConstr(p[g, t] - p[g, prev_t] <= generator_data.loc[g, 'Ramping_Rate'])
    # min-up-time constraint
    for g in G:
        for t_idx, t in enumerate(daily_T):
            max_t_in_day = daily_T[-1]
            tau_range = pd.date_range(
            start=t, 
            end=min(t + pd.Timedelta(hours=generator_data.loc[g, 'Min_Up_Time'] - 1), max_t_in_day), 
            freq='H'
        )
            # if it is the first hour of the day
            if t_idx == 0:
                if day > 0:
                    prev_day_last_t = daily_time_blocks[day - 1][-1]
                    for tau in tau_range:
                        model.addConstr(
                            -u_values[(g, prev_day_last_t)] + u[g, t] - u[g, tau] <= 0
                        )
                else:
                    continue
            else:
                # if not the first hour of the day
                prev_t = daily_T[t_idx - 1]
                for tau in tau_range:
                    model.addConstr(
                        -u[g, prev_t] + u[g, t] - u[g, tau] <= 0
                    )

    # min-down-time constraint
    for g in G:
        for t_idx, t in enumerate(daily_T):
            max_t_in_day = daily_T[-1]
            tau_range = pd.date_range(
            start=t, 
            end=min(t + pd.Timedelta(hours=generator_data.loc[g, 'Min_Down_Time'] - 1), max_t_in_day), 
            freq='H'
        )
            # if it is the first hour of the day
            if t_idx == 0:
                if day > 0:
                    prev_day_last_t = daily_time_blocks[day - 1][-1]
                    for tau in tau_range:
                        model.addConstr(
                            u_values[(g, prev_day_last_t)] - u[g, t] + u[g, tau] <= 0
                        )
                else:
                    continue
            else:
                # if not the first hour of the day
                prev_t = daily_T[t_idx - 1]
                
                for tau in tau_range:
                    model.addConstr(
                        u[g, prev_t] - u[g, t] + u[g, tau] <= 0
                    )

    # solve the model
    model.optimize()
    if model.status == GRB.OPTIMAL:
        
        for g in G:
            for t in daily_T:
                u_values[(g, t)] = u[g, t].X
                p_values[(g, t)] = p[g, t].X
                print(f"Generator {g} at time {t}: Production = {p[g, t].X}, Status = {u[g, t].X}")
    else:
        print("No optimal solution found.")
    # find if it is feasibility
# # 检查是否找到可行解
#     if model.status == GRB.OPTIMAL:
#         print("Optimal solution found.")
        
#         # 输出生成机的生产量和状态
#         for g in G:
#             for t in daily_T:
#                 print(f"Generator {g} at time {t}: Production = {p[g, t].X}, Status = {u[g, t].X}")

#         # 输出 p_nt 变量的值
#         for n in N:
#             for t in daily_T:
#                 print(f"p_nt[{n}, {t}] = {p_nt[n, t].X}")
        
#         # 输出 s 变量的值（启动成本）
#         for g in G:
#             for t in daily_T:
#                 print(f"s[{g}, {t}] = {s[g, t].X}")

#         # 输出 epsilon 和 delta 变量的值
#         for n in N:
#             for t in daily_T:
#                 print(f"epsilon[{n}, {t}] = {epsilon[n, t].X}")
#                 print(f"delta[{n}, {t}] = {delta[n, t].X}")

#         # 输出 u_values 和 p_values 的字典，方便在下一天继续使用
#         for g in G:
#             for t in daily_T:
#                 u_values[(g, t)] = u[g, t].X
#                 p_values[(g, t)] = p[g, t].X

#         # 输出每日最小成本
#         daily_min_cost = model.ObjVal
#         daily_min_costs.append(daily_min_cost)
#         print(f"Minimum cost for day  {day + 1}: {daily_min_cost}")

#     else:
#         print("No optimal solution found.")



    

TypeError: 'Timestamp' object is not iterable

## Optimization For one day

In [ ]:
G = ["G1", "G2", "G3"] 
T = dates
daily_time_blocks = [T[i:i+24] for i in range(0, len(T), 24)]

N = ["N1", "N2", "N3" , "N4" , "N5", "N6"] 
L = ["L1", "L2", "L3" , "L4" , "L5", "L6", "L7"]


# D_nt_df.set_index(['Timestamp', 'N'], inplace=True)
B = [
    [0, -0.68197, -0.65018, -0.48266, -0.51445, -0.63472],
    [0, 0.146043, -0.75311, -0.22164, -0.32249, -0.70405],
    [0, -0.31803, -0.34982, -0.51734, -0.48555, -0.36528],
    [0, 0.17199, 0.102932, -0.26102, -0.19196, 0.069337],
    [0, -0.14604, -0.24689, 0.221642, -0.67751, -0.29595],
    [0, -0.14604, -0.24689, 0.221642, 0.322485, -0.29595],
    [0, 0.146043, 0.246886, -0.22164, -0.32249, -0.70405]
]
max_production = [220, 100, 70]
transmission_capacity = [200, 100, 100, 100, 100, 100, 100]
min_down_time = [4,2,1]
min_production = [100, 10, 10]
min_up_time = [4,3,1]
production_cost = [20.655, 16.7, 22.545]
ramping_rate = [55, 50, 20]
start_up_cost = [900,550,170]
generator_data = pd.DataFrame({
    'Max_Production': max_production,
    'Min_Down_Time': min_down_time,
    'Min_Production': min_production,
    'Min_Up_Time': min_up_time,
    'Production_Cost': production_cost,
    'Ramping_Rate': ramping_rate,
    'Start_Up_Cost': start_up_cost
}, index=G)
B_df = pd.DataFrame(B, index=L, columns=N)
transmission_capacity_df = pd.DataFrame({
    'Transmission_Capacity': transmission_capacity
}, index=L)

Load_1 = merged_data['Hourly_Load_1']
Load_2 = merged_data['Hourly_Load_2']
Load_3 = merged_data['Hourly_Load_3']
P_W1 = merged_data['power_W1']
P_W2 = merged_data['power_W2']


u_values = {}
p_values = {}
daily_min_costs = []

day = 0  
daily_T = daily_time_blocks[0]
model = Model(f"DailyOptimization_Day{day+1}")

M = 1000000

# p, p_nt
p = model.addVars(G, daily_T, lb=0,name="p")
p_nt = model.addVars(N, daily_T, lb=0, name="p_nt")
u = model.addVars(G, daily_T, vtype=GRB.BINARY, name="u")    # on/off
# u = model.addVars(G, daily_T, lb=0, ub = 1, name="u")
s = model.addVars(G, daily_T, lb=0, name="s")                # start-up cost
epsilon = model.addVars(N, daily_T, lb=0, name="epsilon") 
delta = model.addVars(N, daily_T, lb=0, name="delta")   



# Objective function
model.setObjective(
    (quicksum((generator_data.loc[g,'Production_Cost'] * p[g,t] + s[g, t]) for g in G for t in daily_T) + 
    M * quicksum(epsilon[n, t] + delta[n, t] for n in N for t in daily_T)),
    GRB.MINIMIZE
)
# Constraints for generation
for g in G:
    for t in daily_T:
        model.addConstr(generator_data.loc[g, 'Min_Production'] * u[g, t] <= p[g,t])
        model.addConstr(p[g, t] <= generator_data.loc[g, 'Max_Production'] * u[g, t])

# balancing constraint
for t in daily_T:
    model.addConstr(quicksum(p[g,t] for g in G) == quicksum(D_nt_df.loc[t, n] + epsilon[n, t] - delta[n, t] for n in N))

# Transmission limition
for l in L:
    for t in daily_T:
        model.addConstr(
                            quicksum(B_df.loc[l, n] * (p_nt[n, t] - D_nt_df.loc[t, n] - epsilon[n, t] + delta[n, t]) for n in N) >= 
                            -transmission_capacity_df.loc[l, 'Transmission_Capacity']
                        )
        model.addConstr(
            quicksum(B_df.loc[l, n] * (p_nt[n, t] - D_nt_df.loc[t, n] - epsilon[n, t] + delta[n, t]) for n in N) <= 
            transmission_capacity_df.loc[l, 'Transmission_Capacity']
        )
        
# combine p_gt with p_nt
for t in daily_T:
    model.addConstr(p_nt["N1", t] == p["G1", t], name=f"Link_G1_N1_t{t}")
    model.addConstr(p_nt["N2", t] == p["G2", t], name=f"Link_G2_N2_t{t}")
    model.addConstr(p_nt["N6", t] == p["G3", t], name=f"Link_G3_N6_t{t}")
    model.addConstr(p_nt["N3", t] == 0, name=f"Set_N3_Zero_t{t}")
    model.addConstr(p_nt["N4", t] == 0, name=f"Set_N4_Zero_t{t}")
    model.addConstr(p_nt["N5", t] == 0, name=f"Set_N5_Zero_t{t}")


# start-up constraint
for g in G:
    for t_idx, t in enumerate(daily_T):
        if t_idx == 0:
            # if there is data in the previous day use the final data of previous day
            if day > 0:
                prev_day_last_t = daily_time_blocks[day - 1][-1]
                model.addConstr(s[g, t] >= generator_data.loc[g, 'Start_Up_Cost'] * (u[g, t] - u_values[(g, prev_day_last_t)] ))
            # if there is no data in the previous day, no constraint
        else:
            # if t>0 use data in same day
            prev_t = daily_T[t_idx - 1]
            model.addConstr(s[g, t] >= generator_data.loc[g, 'Start_Up_Cost'] * (u[g, t] - u[g, prev_t]))

# ramping rate constraint
for g in G:
    for t_idx, t in enumerate(daily_T):
        if t_idx == 0:
            if day > 0:
                prev_day_last_t = daily_time_blocks[day - 1][-1]
                model.addConstr(p[g, t] - p_values[(g, prev_day_last_t)] >= -generator_data.loc[g, 'Ramping_Rate'])
                model.addConstr(p[g, t] - p_values[(g, prev_day_last_t)] <= generator_data.loc[g, 'Ramping_Rate'])
        else:
            prev_t = daily_T[t_idx - 1]
            print(prev_t)
            model.addConstr(p[g, t] - p[g, prev_t] >= -generator_data.loc[g, 'Ramping_Rate'])
            model.addConstr(p[g, t] - p[g, prev_t] <= generator_data.loc[g, 'Ramping_Rate'])
# min-up-time constraint
for g in G:
    for t_idx, t in enumerate(daily_T):
        max_t_in_day = daily_T[-1]
        tau_range = pd.date_range(
        start=t, 
        end=min(t + pd.Timedelta(hours=generator_data.loc[g, 'Min_Up_Time'] - 1), max_t_in_day), 
        freq='H'
    )
        # if it is the first hour of the day
        if t_idx == 0:
            if day > 0:
                prev_day_last_t = daily_time_blocks[day - 1][-1]
                for tau in tau_range:
                    model.addConstr(
                        -u_values[(g, prev_day_last_t)] + u[g, t] - u[g, tau] <= 0
                    )
            else:
                continue
        else:
            # if not the first hour of the day
            prev_t = daily_T[t_idx - 1]
            for tau in tau_range:
                model.addConstr(
                    -u[g, prev_t] + u[g, t] - u[g, tau] <= 0
                )

# min-down-time constraint
for g in G:
    for t_idx, t in enumerate(daily_T):
        max_t_in_day = daily_T[-1]
        tau_range = pd.date_range(
        start=t, 
        end=min(t + pd.Timedelta(hours=generator_data.loc[g, 'Min_Down_Time'] - 1), max_t_in_day), 
        freq='H'
    )
        # if it is the first hour of the day
        if t_idx == 0:
            if day > 0:
                prev_day_last_t = daily_time_blocks[day - 1][-1]
                for tau in tau_range:
                    model.addConstr(
                        u_values[(g, prev_day_last_t)] - u[g, t] + u[g, tau] <= 0
                    )
            else:
                continue
        else:
            # if not the first hour of the day
            prev_t = daily_T[t_idx - 1]
            
            for tau in tau_range:
                model.addConstr(
                    u[g, prev_t] - u[g, t] + u[g, tau] <= 0
                )

# solve the model
model.optimize()
if model.status == GRB.OPTIMAL:
    
    for g in G:
        for t in daily_T:
            u_values[(g, t)] = u[g, t].X
            p_values[(g, t)] = p[g, t].X
            print(f"Generator {g} at time {t}: Production = {p[g, t].X}, Status = {u[g, t].X}")
else:
    print("No optimal solution found.")

2019-01-01 00:00:00
2019-01-01 01:00:00
2019-01-01 02:00:00
2019-01-01 03:00:00
2019-01-01 04:00:00
2019-01-01 05:00:00
2019-01-01 06:00:00
2019-01-01 07:00:00
2019-01-01 08:00:00
2019-01-01 09:00:00
2019-01-01 10:00:00
2019-01-01 11:00:00
2019-01-01 12:00:00
2019-01-01 13:00:00
2019-01-01 14:00:00
2019-01-01 15:00:00
2019-01-01 16:00:00
2019-01-01 17:00:00
2019-01-01 18:00:00
2019-01-01 19:00:00
2019-01-01 20:00:00
2019-01-01 21:00:00
2019-01-01 22:00:00
2019-01-01 00:00:00
2019-01-01 01:00:00
2019-01-01 02:00:00
2019-01-01 03:00:00
2019-01-01 04:00:00
2019-01-01 05:00:00
2019-01-01 06:00:00
2019-01-01 07:00:00
2019-01-01 08:00:00
2019-01-01 09:00:00
2019-01-01 10:00:00
2019-01-01 11:00:00
2019-01-01 12:00:00
2019-01-01 13:00:00
2019-01-01 14:00:00
2019-01-01 15:00:00
2019-01-01 16:00:00
2019-01-01 17:00:00
2019-01-01 18:00:00
2019-01-01 19:00:00
2019-01-01 20:00:00
2019-01-01 21:00:00
2019-01-01 22:00:00
2019-01-01 00:00:00
2019-01-01 01:00:00
2019-01-01 02:00:00
2019-01-01 03:00:00


## Step 3: Prepare Data for Classification

In [ ]:
features = []
labels = []

for load, wind, status in training_data:
    for t in range(time_periods):
        features.append([load[t], wind[t]])
        labels.append([status[(i, t)] for i in range(len(gen_cost))])  # On/off status for each generator

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

## Step 4: Train and Evaluate Classifiers

In [ ]:
classifiers = []
for i in range(len(gen_cost)):
    clf = LogisticRegression()
    y_train_gen = [y[i] for y in y_train]  # Train labels for generator `i`
    clf.fit(X_train, y_train_gen)
    classifiers.append(clf)

# Evaluate the models
for i, clf in enumerate(classifiers):
    y_test_gen = [y[i] for y in y_test]
    y_pred = clf.predict(X_test)
    print(f'Generator {i+1} Accuracy: {accuracy_score(y_test_gen, y_pred)}')